In [1]:
%load_ext autoreload
%autoreload 2

# Running a single Schwarzschild model

By the end of the notebook, you will have run a Schwarzschild model. This will involve,
1. understanding the format of the configuration file
2. executing commands to create and run a Schwarzschild model
3. plotting some output for this model (kinematic maps)

## Introduction

What is Schwarzschild modelling?

## Setup and reading the configuration file

Let's first import DYNAMITE. For the time being, to do the import, we have to modify out path as follows,

In [2]:
import sys
sys.path.insert(0,'../../')
import dynamite as dyn

Next, let's read the model configuration file. This one file control all of the settings you may wish to vary when running your own Schwarschild models, e.g.

- specifing the components of the gravitational potential
- specifing settings controlling the potential parameters
- specify what type of kinematic data you are providing, e.g.
    - discrete vs continuous,
    - or Gauss Hermite vs Histograms
- the location of the input and output files
- the number of models you want to run

This list is incomplete - for a more detailed description of the configuration file, see the documentation.

The configuration file for this tutorial is
```
NGC6278_config.yaml
```
Open this file in a text editor, alongside this notebook, to see how it is organised. The file is in ``yaml`` format. The basic structure of a yaml files are pairs of keys and values
```
key : value
```
which can be organised into hierarchical levels separated by tabs
```
key_level1:
    keyA_level2 : valueA
    keyB_level2 : valueB  
```
Comments begin with a ``#``. Values can be any type of variable e.g. integers, floats, strings, booleans etc.

To read in the congfiguration file we can use the following command, creating a configuration object which here we call ``c``,

In [3]:
fname = 'NGC6278_config.yaml'
c = dyn.config_reader.Configuration(fname, silent=True)

No previous models have been found:
Making an empty table in AllModels.table


On making this object, some output is printed telling us whether any previous models have been found. Assuming that you're running this tutorial for the first time, then no models will be found and an empty table is created at ``AllModels.table``. This table holds holds information about all the models which have been run so far.

The configuration object ``c`` is structured in a similar way to the the configuration file itself. For example, the configuration file is split into two sections. The top section defines aspects the physical system we wish to model - e.g. the globular cluster, galaxy or galaxy cluster - while the second section contains all other settings we need for running a model - e.g. settings about the orbit integration and input/output options. The two sections are stored in the ``system`` and ``settings`` attributes of the configuration object, respectively,

In [4]:
print(type(c.system))
print(type(c.settings))

<class 'physical_system.System'>
<class 'dynamite.config_reader.Settings'>


The physical system is comprised of components, which are stored in a list,

In [5]:
print(f'cmp_list is a {type(c.system.cmp_list)}')
print(f'cmp_list has length {len(c.system.cmp_list)}')
print('The components are:')
for component in c.system.cmp_list:
    print(f'   ...{component.name}')

cmp_list is a <class 'list'>
cmp_list has length 3
The components are:
   ...black_hole
   ...dark_halo
   ...stars


The following code cell prints information about a particular component,

In [6]:
i = 0
print(f'Information about component {i}:')

# extract component i from the component list
component = c.system.cmp_list[i]

# print the name
print(f'   name =  {component.name}')

# print a list of the names of the parameters of this component
parameters = component.parameters
parameter_names = [par0.name for par0 in parameters]
string = '   has parameters : '
for name in parameter_names:
    string += f'{name} , '
print(string)

# print the type of this component
print(f'   type =  {type(component)}')

# does it contribute to the potential?
print(f'   contributes to the potantial? -  {component.contributes_to_potential}')

Information about component 0:
   name =  black_hole
   has parameters : mass , a , 
   type =  <class 'physical_system.Plummer'>
   contributes to the potantial? -  True


**Exercise 1** : edit the cell to print information about the other components.

For the black hole and dark halo (i.e. components 0 and 1), all information we need to run a model is contained in the configuration file. For the stars - i.e. component 2 - we must provide extra information in the form of input data files. The location of input data files is specified in the configuration file, at
```
settings -> io_settings -> input_directory
```
which takes the value,

In [7]:
c.settings.io_settings['input_directory']

'../../tests/NGC6278/input_data/'

This directory contains four files, and the names of these four files are also specified in the configuration file, at
```
system_components -> stars -> mge_file
system_components -> stars -> kinematics --> datafile
system_components -> stars -> kinematics --> aperturefile
system_components -> stars -> kinematics --> binfile
```
which take the values:
- ``mge.ecvs`` - the Multi Gaussian Expansion (MGE) describing the stellar surface density
- ``gauss_hermite_kins.ecvs`` - the kinematics extracted for this galaxy
- ``aperture.dat`` - information about the spatial apertures/bins using for kinematic extraction
- ``bins.dat`` - information about the spatial apertures/bins using for kinematic extraction

The first two files are in ``Astropy ECSV`` format ([see here](https://docs.astropy.org/en/stable/api/astropy.io.ascii.Ecsv.html)). We have provided test data for one galaxy. A very basic set of instructions for generating your own input data are as follows,

- fit and MGE to a photometric image e.g. using [mge](http://www-astro.physics.ox.ac.uk/~mxc/software/#mge)
- apply voronoi binning to your IFU datacube e.g. using [e.g. vorbin](http://www-astro.physics.ox.ac.uk/~mxc/software/#binning)
- extract kinematics from the binned datacube e.g. using [e.g. PPXF](http://www-astro.physics.ox.ac.uk/~mxc/software/#ppxf)
- make the apertures and binfile files. An example script which has created these files is the file ``generate_input_califa.py`` in [this directory](https://github.com/dynamics-of-stellar-systems/triaxschwarz/tree/sabine/schwpy/data_prepare)

In the future, we will provide more examples of test data, and more detailed instructions for preparing your own data.

As an example of where this data is stored, the MGE can be accessed here,

In [8]:
c.system.cmp_list[2].mge

MGE({'name': None, 'datafile': 'mge.ecvs', 'input_directory': '../../tests/NGC6278/input_data/', 'data': <Table length=6>
   I      sigma      q    PA_twist
float64  float64  float64 float64 
-------- -------- ------- --------
26819.14  0.49416 0.89541      0.0
 2456.39  2.04299 0.79093      0.0
   456.8  2.44313  0.9999      0.0
  645.49   6.5305 0.55097      0.0
   14.73 17.41488  0.9999      0.0
  123.85 21.84711 0.55097      0.0})

while the kinematics are stored at

In [9]:
type(c.system.cmp_list[2].kinematic_data)

list

Note that this object has type ``list``. This is so that - in the future - we could allow a single component to have multiple different sets of kinematics, For now, we only have one set of kinematics, which is the first (and only) entry in the list

In [10]:
type(c.system.cmp_list[2].kinematic_data[0])

kinematics.GaussHermite

We see that this kinemtics object has type  ``GaussHermite``. This has also been specified in the configuration file, under
```
system_components -> stars -> kinematics --> set1 --> type
```
At present ``GaussHermite`` is the only type of kinematics available. In the future, we want to expand this to include other options. The kinemtic data itself can be accessed as follows,

In [11]:
c.system.cmp_list[2].kinematic_data[0].data

vbin_id,v,dv,sigma,dsigma,h3,dh3,h4,dh4
int64,float64,float64,float64,float64,float64,float64,float64,float64
1,-55.1457,2.0968,193.5,2.0798,0.077,0.3,0.0,0.3
2,-72.0331,2.1187,173.297,2.2978,0.1006,0.3,0.0,0.3
3,-70.3214,2.9704,157.694,3.0237,0.0983,0.3,0.0,0.3
4,-59.3916,2.641,179.503,2.6,0.083,0.3,0.0,0.3
5,-42.9252,2.627,203.949,2.4986,0.06,0.3,0.0,0.3
6,-76.6923,4.1897,153.159,4.2396,0.1072,0.3,0.0,0.3
7,-60.8558,2.8168,154.194,3.0038,0.085,0.3,0.0,0.3
8,-45.534,2.6694,184.95,2.801,0.0636,0.3,0.0,0.3
9,-73.7064,3.3263,133.986,3.5988,0.103,0.3,0.0,0.3


## Creating a Schwarzschild model

Our next step will be to create a model. The init method (replace with docblock!) of the Model object (i.e. line 136 of ``dynmaite/model.py`` looks as follows
```
    def __init__(self,
                 system=None,
                 settings=None,
                 parspace=None,
                 executor=None,
                 parset=None):
```
It requires 5 input parameters. We've already met the ``system`` and ``settings`` parameters. Two of the others are internally created when we run the configuration reader,

In [12]:
print(type(c.parspace))
print(type(c.executor))

<class 'parameter_space.ParameterSpace'>
<class 'executor.Local'>


I'll describe ``parspace`` below. To learn more about the ``executor``, read the tutorial/documentation XXXXX about multiprocessing.

The remaining input parameter we need to provide is ``parset``. This is a set of parameters needed to define the gravitational potential of the system. Properties about the parameters are set in the configuration file under the key ``parameters``. A detailed description of all these properties can be found in tutorial/documentaion XXXXX.

For now, let's extract a single parameter set to be able to create a model. For every parameter, in the configuration file we have specified a ``value``. Let's look at these values. To access them, we can loop over ``parspace``, which is a list of all the parameters,

In [13]:
print('Parameter / value :')
for par in c.parspace:
    print(f'   {par.name} = {par.value}')

Parameter / value :
   mass = 6.0
   a = 0.001
   dc = 10.0
   f = 100.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 3.0


This is almost the ``parset`` that we need to make the model. One complication is that some parameters are specified in logarithmic units, i.e. 
```
parameters -> XXX -> logarithmic : True
```
This can be useful for parameters (e.g. masses) for which it makes sense to take logarithmically spaced steps through parameter space. For other parameters (e.g. length scales) linearly spaced steps may be more appropriate. For other types of parameters (e.g. angles) a different spacing may be preferable.

To handle these possibilities, we introduce the concept of ``raw`` parameter values, distinct from the values themselves. All values associated with parameter in the configuration file are given in ``raw`` units. When we step through parameter space, we take linear steps in ``raw`` values. The conversion from raw values to the parameter values is handles by the method
```
Parameter.get_par_value_from_raw_value
```
So to convert the above list from raw values, we can do the following,

In [14]:
print('Parameter / value :')
for par in c.parspace:
    raw_value = par.value
    par_value = par.get_par_value_from_raw_value(raw_value)
    print(f'   {par.name} = {par_value}')

Parameter / value :
   mass = 1000000.0
   a = 0.001
   dc = 10.0
   f = 100.0
   q = 0.54
   p = 0.99
   u = 0.9999
   ml = 3.0


Lastly, we must pass a parameter set to the ``Model`` in the form of a row of an ``Astropy Table``, which we can create as follows,

In [15]:
from astropy.table import Table

t = Table()
for par in c.parspace:
    raw_value = par.value
    par_value = par.get_par_value_from_raw_value(raw_value)
    t[par.name] = [par_value]
# extract 0th - i.e. the only - row from the table
parset = t[0]

print(parset)

   mass     a    dc    f    q    p     u     ml
--------- ----- ---- ----- ---- ---- ------ ---
1000000.0 0.001 10.0 100.0 0.54 0.99 0.9999 3.0


OR... (maybe) move all this detail above about raw units, Astropy tables etc, to a different tutorial, and replace it with the following.

Parameter values are specified in the configuration file. We can extract this set of values in the required format as follows

In [16]:
parset = c.parspace.get_parset()
print(parset)

   mass     a    dc    f    q    p     u     ml
--------- ----- ---- ----- ---- ---- ------ ---
1000000.0 0.001 10.0 100.0 0.54 0.99 0.9999 3.0


This is a row of an astropy Table object. Note that, compared to values specified in the configuration file, parameters which have been specified as logarithmic, i.e.
```
parameters -> XXX -> logarithmic : True
```
have been exponentiated in here. More details can be found in the tutorial/documenation XXXXX.

We can now create our model,

In [17]:
model = dyn.model.LegacySchwarzschildModel(
    system=c.system,
    settings=c.settings,
    parspace=c.parspace,
    executor=c.executor,
    parset=parset)

Nothing happens. But soon it will.

## Running a model 

First, lets setup a directory for the output

In [18]:
model.setup_directories()

This should have create a directory called 

In [19]:
c.settings.io_settings['output_directory']

'NGC6278_output/'

inside of which you should find 
```
NGC6278_output/models/
```
inside of which a unique directory for *this* model has been created, called

In [20]:
model.get_model_directory()

'NGC6278_output/models/mass1000000.00a0.00dc10.00f100.00q0.54p0.990u0.9999/ml3.00/'

The next step is to calculate the orbit library. This step will take a few minutes,

In [21]:
model.get_orblib()

FileNotFoundError: [Errno 2] No such file or directory: '../../tests/NGC6278/input_data/kin_data.dat'

Having calculated an orbit library, we now need to find out which orbits are any good i.e. are useful in reproducing our observations. This is an Non-Negative Least Squares (NNLS) optimization problem, which is done as follows,

In [ ]:
model.get_weights()

Congratulations! You have run your first Schwarzschild model using DYNAMITE. The chi$^2$ of this model is,

In [ ]:
model.chi2

Is that good? I don't know! To find out, we'll have to run more models and compare their chi$^2$ values. For information about this, see the tutorial running_a_grid_of_models.

For now, let's dive deeper into this one model that we *have* run.

## Plot the Models

In [ ]:
plotter = dyn.plotter.Plotter(system=c.system,
                              settings=c.settings,
                              parspace=c.parspace,
                              all_models=c.all_models)

In [ ]:
figure = plotter.plot_kinematic_maps(model)

## Exercises

Stuff to try out:
- change the number of Gauss Hermite coeffients used: does the problem go away?
- can you find other LOSVDs problematic for the GH expansion?
- what is the total weight of orbits which go through aperture 0?
- what is the total weight of orbits which have negative velocity in apeture 0?
- plot the observed/model/model-GH LOSVDs of the whole galaxy (i.e. combine all apertures together)
- run another model with a larger orbit library
- run another model with a larger orbit library